## Import relevant modules

In [320]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mt
import seaborn as sns
import numpy as np
import datetime as dt
import math
import psycopg2
import statsmodels.api as sm

## Query the MIMIC database for relevant patient information

In [266]:
conn = psycopg2.connect(dbname="mimic", user="pradeepbandaru", password="RasRafB2H")

# calculate patient age at admission and store along with select columns from icustays table
icustays_query = """SELECT ie.subject_id, ie.hadm_id, ie.icustay_id,
ie.intime, ie.outtime, ie.first_careunit,
ROUND((cast(ie.intime as date) - cast(pat.dob as date))/365.242, 2) AS age_admit
FROM mimiciii.icustays ie
INNER JOIN mimiciii.patients pat
ON ie.subject_id = pat.subject_id;
"""

# calculate patient age at death and store along with select columns from admissions table
admissions_query = """SELECT ad.subject_id, ad.hadm_id, ad.deathtime,
ad.insurance, ad.language, ad.religion, ad.marital_status, ad.admission_type,
case
    when ad.ethnicity like '%WHITE%' then 'WHITE'
    when ad.ethnicity like '%BLACK%' then 'BLACK'
    when ad.ethnicity like '%HISPANIC%' then 'HISPANIC'
    when ad.ethnicity like '%ASIAN%' then 'ASIAN'
    else 'OTHER' end as ethnicity,
ad.hospital_expire_flag,
ROUND((cast(ad.deathtime as date) - cast(pat.dob as date))/365.242, 2) AS age_death
FROM mimiciii.admissions ad
INNER JOIN mimiciii.patients pat
ON ad.subject_id = pat.subject_id;
"""

# select columns from diagnoses_icd table
diagnoses_icd_query = """SELECT di.subject_id, di.hadm_id, di.icd9_code  
FROM mimiciii.diagnoses_icd di"""

# select columns from patients table
patients_query = """SELECT pat.subject_id, pat.gender FROM mimiciii.patients pat"""

# execute SQL queries and store in pandas dataframes
icustays_table = pd.read_sql_query(icustays_query,conn)
admissions_table = pd.read_sql_query(admissions_query,conn)
diagnoses_icd_table = pd.read_sql_query(diagnoses_icd_query,conn)
patients_table = pd.read_sql_query(patients_query,conn)

In [267]:
# combine all tables into one master table per patient
dfs = [icustays_table, admissions_table, diagnoses_icd_table, patients_table]
master = reduce(lambda left,right: pd.merge(left,right,on='subject_id'), dfs)
master = master.drop_duplicates(subset=['subject_id', 'hadm_id'])

# get rid of unwanted columns
cols = ['hadm_id_y', 'hadm_id', 'intime', 'outtime', 'deathtime']
master.drop(cols, inplace=True, axis=1)

## Filter master patient table by disease-specific ICD9 codes

In [315]:
#master.head(15)
#print df_final.shape

# filter the master table by patients who have died in the hospital
#master2 = master.loc[master['hospital_expire_flag'] == 1]

#filter the master table by icd9 codes that correspond to AKI or MI
#MI:
#master2 = master.loc[master['icd9_code'].isin(['41011', '41091', '412', '4139', '41401'])]

#AKI:
master2 = master.loc[master['icd9_code'].isin(['5939', '5849'])]
master2.head()
#master2.shape
#make some plots from the master table

,subject_id,hadm_id_x,icustay_id,first_careunit,age_admit,insurance,language,religion,marital_status,admission_type,ethnicity,hospital_expire_flag,age_death,icd9_code,gender
3544,344,176203,238865,MICU,58.18,Private,None,CATHOLIC,WIDOWED,EMERGENCY,WHITE,1,58.19,5849,F
720503,638,176623,281712,CCU,84.23,Medicare,None,CATHOLIC,WIDOWED,EMERGENCY,WHITE,0,NaN,5849,F
723077,843,141809,218680,MICU,45.35,Private,None,CATHOLIC,SINGLE,EMERGENCY,OTHER,0,NaN,5849,M
730627,948,107027,205442,MICU,28.54,Private,None,CATHOLIC,MARRIED,EMERGENCY,WHITE,0,NaN,5849,F
736175,1339,104951,292224,MICU,66.19,Private,ENGL,JEWISH,MARRIED,EMERGENCY,WHITE,0,NaN,5849,F


## Train a regression model

In [327]:
# remove missing data
#master2 = master.dropna(axis=0)

# create dummy variables for parameters
dummy_list = ['first_careunit', 'marital_status', 'ethnicity', 'admission_type', 'insurance']
dummy_frames = [pd.get_dummies(master2[x]) for x in dummy_list]
dummy_frames = pd.concat(dummy_frames, axis=1)

In [328]:
master2['gender'] = (master2.gender == 'F').astype(str)
master3 = pd.concat([master2, dummy_frames], axis=1)
master3['intercept'] = 1.0
#master3['MICU'] = np.where(master3.MICU == 1, 1, master3.MICU)
master3 = master3.reset_index(drop=True).T.drop_duplicates().T.set_index(master3.index)
train_cols = ['intercept','age_admit', 'CSRU', 'MICU', 'SICU', 'CCU', 
              'DIVORCED', 'MARRIED', 'SEPARATED', 'WIDOWED', 'SINGLE', 
              'ASIAN', 'BLACK', 'HISPANIC', 'WHITE', 
              'EMERGENCY', 'URGENT',
              'Medicaid', 'Medicare', 'Private', 'Government']

In [329]:
logit = sm.Logit(master3.gender, master3[train_cols])
result = logit.fit()
result.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).